In [1]:
import numpy as np
import pandas as pd

In [21]:
df = pd.read_excel("指数优化.xlsx")

In [52]:
df.head()

,基金代码,基金名称,公司名称,成立年限,获配新股个数,获配金额(万元),打新收益(万元),平均净值(万元),打新收益贡献,区间总净值增长,...,成立至今年化收益,abs最大回撤,Sortino,PAIndex,Stability,NSContriRet,NorSortino,NorPAIndex,NorStab,NordNCR
0,009428.OF,鹏扬景沣六个月持有A,鹏扬基金管理有限公司,0.983562,282,2560.3470,2628,137602,0.0191,0.0731,...,0.082723,2.260884,0.262100,0.116097,0.024971,0.261286,0.887625,3.019922,0.502030,0.326143
1,519050.OF,海富通安颐收益A,海富通基金管理有限公司,8.191781,367,2693.2740,3899,62298,0.0626,0.1096,...,0.087861,3.716803,0.200488,0.059485,0.008168,0.571168,0.515301,1.105594,1.399324,1.107095
2,010401.OF,新华安康多元收益一年持有A,新华基金管理股份有限公司,0.635616,173,857.7897,1724,105603,0.0163,0.0279,...,0.052307,3.897732,0.166139,0.050364,0.002447,0.584229,0.307727,0.797165,1.704767,1.140012
3,007884.OF,易方达恒盛3个月定开,易方达基金管理有限公司,1.915068,275,2319.4030,2663,80948,0.0329,0.1249,...,0.082640,2.997003,0.359843,0.063825,0.013883,0.263411,1.478291,1.252348,1.094129,0.331498
4,005984.OF,兴业聚华A,兴业基金管理有限公司,1.378082,404,3075.8070,4051,101937,0.0397,0.1531,...,0.132145,2.715156,0.269640,0.067702,0.011352,0.259308,0.933190,1.383431,1.229260,0.321157


In [109]:
def optifunc(x):
    """x is an array of initial values representing the weights of each parameters"""
    score = []
    for l in range(len(df)):
        score.append(df['NorsucRate'][l]*x[0] + df['NorAskVal'][l]*x[1] - df['NorBid'][l]*x[2])
    
    max_index = []
    for i in range(20):
        index = score.index(max(score))
        max_index.append(index)
        score[index] = 0
    
    para = [0,0,0,0]
    for ind in max_index:
        para[0] += df['CompanyVal'][ind]
        para[1] += df['Ask'][ind]
        para[2] += df['SucRate'][ind]
        para[3] += df['BidPrice'][ind]
    para = [x/20 for x in para]
    
    result = (para[0] - 3049.01) / para[0] + (para[1] - 21220) / para[1] + (para[2] - 0.919) / para[2] + (1.004 - para[3]) / para[3]
    return result

In [82]:
init = [0.3,0.4,0.2,0.1]

In [83]:
optifunc(init)

[0.3, 0.4, 0.2, 0.1]
0.8334450811258562


-0.8334450811258562

In [84]:
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
x = [0.3,0.4,0.2,0.1]
bnds = tuple((0,1) for y in x)

In [85]:
from scipy.optimize import minimize

In [86]:
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

In [92]:
sol  = minimize(optifunc, x,method='SLSQP',jac=rosen_der, bounds=bnds ,options={'ftol': 0.1, 'disp': True},constraints=cons)
print(sol)

[0.3 0.4 0.2 0.1]
0.8334450811258562
[1.00000000e+00 1.36838318e-11 0.00000000e+00 1.28913547e-12]
-2.5536985599120876
[0.62791836 0.21261808 0.10630904 0.05315452]
0.46510517390490125
[0.46143563 0.30775107 0.15387554 0.07693777]
0.6877442421783179
[0.37971657 0.35444768 0.17722384 0.08861192]
0.7446764200861052
[0.33925003 0.37757141 0.1887857  0.09439285]
0.7992719330468597
[0.31939008 0.38891996 0.19445998 0.09722999]
0.7992719330468597
[0.30946295 0.3945926  0.1972963  0.09864815]
0.7946068424268065
[0.30447583 0.39744238 0.19872119 0.0993606 ]
0.8334450811258562
[0.30223791 0.39872119 0.1993606  0.0996803 ]
0.8334450811258562
[0.30111896 0.3993606  0.1996803  0.09984015]
0.8334450811258562
[0.30055948 0.3996803  0.19984015 0.09992007]
0.8334450811258562
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8334450811258562
            Iterations: 1
            Function evaluations: 12
            Gradient evaluations: 1
     fun: -0.83344508

In [143]:
suc = np.linspace(0.3,0.4,num = 5)
ask = np.linspace(0.3,0.4,num = 5)
val = np.linspace(0.1,0.2,num = 5)

comb = []
result = []
for s in suc:
    for a in ask:
        for v in val:
            b = 1 - s - a - v
            x0 = [s,a,v,b]
            x = optifunc(x0)
            comb.append(x0)
            result.append(x)

In [144]:
inin = result.index(max(result))

In [40]:
def Rankopt(x):
    """x is an array of initial values representing the weights of each parameters"""
    score = []
    for l in range(len(df)):
        score.append(df['NorTA'][l]*x[0] + df['NorNCR'][l]*x[1] + df['NorSP'][l]*x[2] + df['Nor'][l]*x[3])
    
    rank_index = []
    for i in range(len(df)):
        index = score.index(max(score))
        rank_index.append(index)
        score[index] = -100
    
    rank_index_total = [0 for x in range(len(df))]
    for x in range(len(rank_index)):
        rank_index_total[rank_index[x]] = x
    
    """
    rank_index_top10 = [0 for x in range(len(df))]
    for x in range(len(rank_index)):
        if x >= 9:
            rank_index_total[rank_index[x]] += 1
    
    rank_index_top20 = [0 for x in range(len(df))]
    for x in range(len(rank_index)):
        if x >= 19:
            rank_index_total[rank_index[x]] += 1
    
    rank_index_top30 = [0 for x in range(len(df))]
    for x in range(len(rank_index)):
        if x >= 29:
            rank_index_total[rank_index[x]] += 1
    
    para = [0,0,0,0]
    for ind in rank_index:
        para[0] += df['CompanyVal'][ind]
        para[1] += df['Ask'][ind]
        para[2] += df['SucRate'][ind]
        para[3] += df['BidPrice'][ind]
    para = [x/20 for x in para]
    
    result = (para[0] - 3049.01) / para[0] + (para[1] - 21220) / para[1] + (para[2] - 0.919) / para[2] + (1.004 - para[3]) / para[3]
    """
    
    return rank_index_total

In [22]:
df.head()

,基金代码,基金名称,公司名称,成立年限,获配新股个数,获配金额(万元),打新收益(万元),平均净值(万元),打新收益贡献,区间总净值增长,...,成立至今年化收益,abs最大回撤,Sortino,PAIndex,Stability,NSContriRet,NorSortino,NorPAIndex,NorStab,NordNCR
0,008456.OF,招商瑞阳A,招商基金管理有限公司,1.547945,402,4250.228,4558,41172,0.1107,0.2661,...,0.185311,3.307152,0.478306,0.079368,0.003898,0.416009,2.194168,1.394316,1.617273,0.716070
1,000110.OF,金鹰元安A,金鹰基金管理有限公司,8.219178,394,2716.938,3392,45831,0.0740,0.2282,...,0.086879,1.500506,0.321057,0.147347,0.005381,0.324277,1.243903,2.833405,1.569752,0.484890
2,151002.OF,银河收益,银河基金管理有限公司,18.019178,423,3630.405,4438,63869,0.0695,0.2036,...,0.108117,0.725348,0.216091,0.270637,0.002100,0.341356,0.609589,5.443413,1.674911,0.527931
3,005652.OF,国富天颐A,国海富兰克林基金管理有限公司,3.364384,419,3291.271,4124,60840,0.0678,0.1957,...,0.097945,2.816268,0.327876,0.067921,0.006918,0.346449,1.285113,1.151989,1.520474,0.540766
4,006973.OF,太平睿盈A,太平基金管理有限公司,2.369863,415,3665.247,4518,36256,0.1246,0.1893,...,0.130076,2.293202,0.269028,0.080159,0.004028,0.658214,0.929492,1.411058,1.613110,1.326466


In [23]:
def Rankopt_FI(x):
    """x is an array of initial values representing the weights of each parameters"""
    score = []
    for l in range(len(df)):
        score.append(df['NorSortino'][l]*x[0] + df['NorPAIndex'][l]*x[1] + df['NorStab'][l]*x[2] + df['NordNCR'][l]*x[3])
    
    rank_index = []
    for i in range(len(df)):
        index = score.index(max(score))
        rank_index.append(index)
        score[index] = -100
    
    rank_index_total = [0 for x in range(len(df))]
    for x in range(len(rank_index)):
        rank_index_total[rank_index[x]] = x
    
    return rank_index_total

In [10]:
def Rankopt_Ind(x):
    """x is an array of initial values representing the weights of each parameters"""
    score = []
    for l in range(len(df)):
        score.append(df['NorNCR'][l]*x[0] + df['NorIndDif'][l]*x[1] + df['NorIndEx'][l]*x[2] - df['NorFee'][l]*x[3])
    
    rank_index = []
    for i in range(len(df)):
        index = score.index(max(score))
        rank_index.append(index)
        score[index] = -100
    
    rank_index_total = [0 for x in range(len(df))]
    for x in range(len(rank_index)):
        rank_index_total[rank_index[x]] = x
    
    return rank_index_total

In [24]:
TA = np.linspace(0,0.5,num = 8)
NCR = np.linspace(0,0.5,num = 8)
SP = np.linspace(0,0.5,num = 8)
AP = np.linspace(0,0.5,num = 8)

"Total chart"
TC = [[] for x in range(len(df))]
scores = []
for s in TA:
    for a in NCR:
        for v in SP:
            for b in AP:
                x0 = [s,a,v,b]
                total = Rankopt_FI(x0)
                for x in range(len(TC)):
                    TC[x].append(total[x])
                #scores.append(diff)

In [25]:
#avg_score = np.mean(scores)
good_ind = []
for x in range(len(scores)):
    if scores[x] > 0:
        good_ind.append(x)

In [26]:
len(good_ind)

0

In [27]:
avgTC = []
std = []
for x in TC:
    avgTC.append(np.mean(x))
    std.append(np.std(x))

In [28]:
min_avgTC = []
minini = []
sttd = []
for i in range(len(df)):
    mini = min(avgTC)
    index = avgTC.index(mini)
    min_avgTC.append(index)
    minini.append(mini)
    sttd.append(std[index])
    avgTC[index] = 200

In [29]:
for x in range(len(minini)):
    print(df['基金名称'][min_avgTC[x]],'   avg_rank:',round(minini[x]+1,2), '   avg_std:',round(sttd[x],2))

工银聚和一年定开A    avg_rank: 4.53    avg_std: 6.12
国寿安保稳泰A    avg_rank: 8.5    avg_std: 9.34
华夏睿磐泰茂A    avg_rank: 8.61    avg_std: 2.19
银河收益    avg_rank: 9.01    avg_std: 14.17
招商瑞阳A    avg_rank: 10.64    avg_std: 4.17
鹏华尊惠18个月A    avg_rank: 10.88    avg_std: 5.26
中加科盈A    avg_rank: 11.26    avg_std: 14.79
金鹰元安A    avg_rank: 11.82    avg_std: 8.34
前海开源裕和A    avg_rank: 13.96    avg_std: 15.88
华夏睿磐泰利A    avg_rank: 14.05    avg_std: 4.76
国寿安保稳荣A    avg_rank: 14.11    avg_std: 15.94
太平睿盈A    avg_rank: 14.47    avg_std: 4.27
招商瑞信稳健配置A    avg_rank: 16.31    avg_std: 4.68
东方红优质甄选一年持有    avg_rank: 16.6    avg_std: 18.1
中加科丰价值精选    avg_rank: 18.29    avg_std: 9.96
中欧瑾利A    avg_rank: 20.84    avg_std: 22.6
国寿安保稳吉A    avg_rank: 21.66    avg_std: 3.4
华泰柏瑞景利A    avg_rank: 21.77    avg_std: 13.86
创金合信鑫祺A    avg_rank: 21.88    avg_std: 8.74
鹏华安惠A    avg_rank: 22.11    avg_std: 7.91
华夏睿磐泰兴    avg_rank: 23.7    avg_std: 5.82
国富天颐A    avg_rank: 24.55    avg_std: 5.32
南方安康    avg_rank: 24.95    avg_std: 8.28
长

In [30]:
newdf = df.iloc[min_avgTC,:]

In [31]:
newdf['avg_std'] = sttd
newdf['avg_rank'] = [x + 1 for x in minini]
newdf

<ipython-input-31-a9566fd2b694>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['avg_std'] = sttd
<ipython-input-31-a9566fd2b694>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['avg_rank'] = [x + 1 for x in minini]


,基金代码,基金名称,公司名称,成立年限,获配新股个数,获配金额(万元),打新收益(万元),平均净值(万元),打新收益贡献,区间总净值增长,...,Sortino,PAIndex,Stability,NSContriRet,NorSortino,NorPAIndex,NorStab,NordNCR,avg_std,avg_rank
42,009031.OF,工银聚和一年定开A,工银瑞信基金管理有限公司,1.243836,350,3141.351,3105,62089,0.0500,0.1201,...,1.139746,0.067563,0.001687,0.416320,6.191283,1.144406,1.688130,0.716853,6.118545,4.530029
8,004772.OF,国寿安保稳泰A,国寿安保基金管理有限公司,3.958904,341,2429.523,3145,56049,0.0561,0.1679,...,0.826593,0.057531,0.001064,0.334127,4.298884,0.932033,1.708106,0.509715,9.340247,8.495605
13,004720.OF,华夏睿磐泰茂A,华夏基金管理有限公司,3.668493,396,3446.570,4337,51537,0.0841,0.1656,...,0.356302,0.112841,0.002107,0.507850,1.456892,2.102927,1.674697,0.947524,2.185606,8.614746
2,151002.OF,银河收益,银河基金管理有限公司,18.019178,423,3630.405,4438,63869,0.0695,0.2036,...,0.216091,0.270637,0.002100,0.341356,0.609589,5.443413,1.674911,0.527931,14.169220,9.007080
0,008456.OF,招商瑞阳A,招商基金管理有限公司,1.547945,402,4250.228,4558,41172,0.1107,0.2661,...,0.478306,0.079368,0.003898,0.416009,2.194168,1.394316,1.617273,0.716070,4.171607,10.639893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,519050.OF,海富通安颐收益A,海富通基金管理有限公司,8.194521,367,2693.274,3899,62298,0.0626,0.1096,...,0.200488,0.028860,0.072061,0.571168,0.515301,0.325074,-0.567359,1.107095,6.566121,63.270752
67,009499.OF,景顺长城安鑫回报A,景顺长城基金管理有限公司,1.073973,229,1153.581,1807,53149,0.0340,0.0340,...,0.092244,0.017101,0.114912,1.000000,-0.138827,0.076138,-1.940765,2.187821,13.245150,63.792480
54,002813.OF,博时颐泰A,博时基金管理有限公司,5.120548,255,2137.403,2420,57931,0.0418,0.0752,...,0.123740,0.041859,0.096674,0.555851,0.051506,0.600258,-1.356229,1.068494,6.140597,66.440674
61,010508.OF,博时鑫康A,博时基金管理有限公司,0.750685,207,1278.822,1937,77227,0.0251,0.0446,...,0.114167,0.028589,0.116808,0.562780,-0.006341,0.319336,-2.001521,1.085957,4.776260,68.773926


In [89]:
#newdf.to_excel('EquityPlusRank_result.xlsx')

In [32]:
newdf.to_excel('FIPlusRank_result.xlsx')

In [33]:
#newdf.to_excel('IndexPlusRank_result.xlsx')